# Langchain

learn main components with small mini projects

### Models Components

In [ ]:
# Models: Close source / Open source
# topics:
# - Text generation
# - Temperature, top_p, top_k
# - invoke and stream

# Embedding models: create embeddings from text
# task:
# - Create embeddings from text documents
# - Store embeddings in vector databases
# - Similarity search and calculate cosine similarity

In [ ]:
# Embedding task

# ollama pull embeddinggemma
# check ollama model using "ollama serve" in cmd
from langchain.embeddings.ollama import OllamaEmbeddings

data = """ 
What is Machine Learning?
Machine learning is a branch of artificial intelligence that enables algorithms to uncover hidden patterns within datasets. 
It allows them to predict new, similar data without explicit programming for each task. 
Machine learning finds applications in diverse fields such as image and speech recognition, 
natural language processing, recommendation systems, fraud detection, portfolio optimization, and automating tasks.

Types of Machine Learning
Machine learning algorithms can be broadly categorized into three main types based on their learning approach and the nature of the data they work with.

Supervised Learning
Involves training models using labeled datasets. Both input and output variables are provided during training.
The aim is to establish a mapping function that predicts outcomes for new, unseen data.
Common applications include classification, regression, and forecasting.

Unsupervised Learning
Works with unlabeled data where outputs are not known in advance.
The model identifies hidden structures, relationships, or groupings in the data.
Useful for clustering, dimensionality reduction, and anomaly detection.
Focuses on discovering inherent patterns within datasets.

Reinforcement Learning
Based on decision-making through interaction with an environment.
An agent performs actions and receives rewards or penalties as feedback.
The goal is to learn an optimal strategy that maximizes long-term rewards.
Widely applied in robotics, autonomous systems, and strategic game playing.
"""

embeddings = OllamaEmbeddings(model="embeddinggemma")


# create chunked documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([data])

print(f"Created {len(docs)} documents")
print(f"first document: {docs[0]}")

# create embeddings
doc_embeddings = embeddings.embed_documents([doc.page_content for doc in docs])
print(f"\nCreated {len(doc_embeddings)} embeddings")

# calculate cosine similariy between embeddings (numpy)
import numpy as np
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

similarity = cosine_similarity(doc_embeddings[0], doc_embeddings[1])
print(f"\nCosine similarity between first two embeddings: {similarity}")

Created 4 documents
first document: page_content='What is Machine Learning?
Machine learning is a branch of artificial intelligence that enables algorithms to uncover hidden patterns within datasets. 
It allows them to predict new, similar data without explicit programming for each task. 
Machine learning finds applications in diverse fields such as image and speech recognition, 
natural language processing, recommendation systems, fraud detection, portfolio optimization, and automating tasks.'

Created 4 embeddings

Cosine similarity between first two embeddings: 0.679702416172827


In [ ]:
### notes

"""
text input > chunks            :    RecusriveCharacterTextSplitter(), splitter.from_documents(LIST) 

chunks     > embedding vector  :    embedding.embed_documents([doc.page_content for doc in docs])           # embed contents of each document
"""

### Prompt Components

In [7]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model = "phi")

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an Expert assistant"),
    ("human", "Give me a short description of Langchain framework")
])

chain = prompt | llm

chain.invoke({})

AIMessage(content=' Langchain is a language-independent software framework for building and running microservices. It allows developers to write code in any programming language and deploy it as separate services on a single machine or across multiple machines. Langchain supports popular languages such as Java, Python, C#, Go, and Ruby. Its architecture includes a runtime component that handles the execution of the code and a messaging system for communication between services.\n\nLangchain is designed to enable microservices development and deployment in a fast and scalable manner. It allows developers to create complex applications by breaking them down into smaller, independent services that can be developed and tested independently. Langchain also provides features such as security, observability, and fault tolerance for highly scalable microservices architectures.\n\nOverall, Langchain is an innovative framework for building and running microservices that enables developers to del

🧠 Mini Projects

* Save a pdf into a vectorDB

* Chatbot — Built a simple chatbot using ChatPromptTemplate and message history.

* Research Paper Summarizer — Created a summarization tool that accepts a research paper as input and outputs a concise summary using prompt templates.

In [ ]:
## Save a pdf into a vectorDB

In [15]:
%pip install -qU langchain-community pypdf
%pip install -qU langchain-community faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install "langchain==0.3.27"

In [ ]:
%pip show langchain
%pip show langchain-core

In [ ]:
"""
#  why this code doesn't work?
"""

from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


input_pdf_path = r"data\event_pdfs\Event-Based_Vision_A_Survey.pdf"
loader = PyPDFLoader(input_pdf_path)
docs = loader.load()

print(f"number of documents: {len(docs)}")
# print(docs[1])

# now, we have each page, let's create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=100)
chunks = splitter.split_text(docs)
print(f"number of chunks: {len(chunks)}")

In [ ]:
# the main issue of the previous code is
# docs is a list of documents
# split_text:  take "text" as the input not a list
# split_documents: take a list of document

In [ ]:
"""
note: this code took some minutes

        reason: for each chunk it call embedding model, for example if there are 459 chunks >> 459 embedding calls

      it's better to use persist_directory to save the embeddings (otherwise it calculates at every run!)
"""
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings.ollama import OllamaEmbeddings



input_pdf_path = r"data\event_pdfs\Event-Based_Vision_A_Survey.pdf"
loader = PyPDFLoader(input_pdf_path)
docs = loader.load()

print(f"number of documents: {len(docs)}")

# now, we have each page, let's create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"number of chunks: {len(chunks)}")

# embedding and store
embeddings = OllamaEmbeddings(model="embeddinggemma")
vectordb   = FAISS.from_documents(chunks, embeddings)

In [ ]:
## Chroma instead of FAISS

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings.ollama import OllamaEmbeddings



input_pdf_path = r"data\event_pdfs\Event-Based_Vision_A_Survey.pdf"
loader = PyPDFLoader(input_pdf_path)
docs = loader.load()

print(f"number of documents: {len(docs)}")

# now, we have each page, let's create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"number of chunks: {len(chunks)}")

# embedding and store
embeddings = OllamaEmbeddings(model="embeddinggemma")
vectordb   = Chroma.from_documents(chunks[:10], embeddings)

number of documents: 27
number of chunks: 459


In [11]:
# Now let's see what is stored in memory
vectordb._collection.peek()

{'ids': ['d9785822-975a-45d1-9280-e971d9bd569c',
  'd6723876-bda3-4982-9209-f93f58a40868',
  'd2c03d6c-01b9-4a9b-aeb7-b027f3473db2',
  '8a2ae3e0-56fb-45ca-840a-7f27720660f0',
  'dbfcb9a8-f718-4caa-91f9-88bdb3ed93af',
  'eae46d1f-93fe-4eea-bb95-4f92da617a4c',
  'dbd64686-21f0-4004-b737-393cf4376176',
  'fb610ada-4375-4f5d-b4ee-2a5781eaf7e0',
  'bd0ce911-1028-421f-9ef1-58f9d51fbb63',
  '3dacdf6c-58d9-4649-9252-380e9f07a92c'],
 'embeddings': array([[-0.07905253, -0.03372795, -0.03010053, ..., -0.03979218,
         -0.00469344, -0.02931392],
        [-0.04156356, -0.02531246, -0.02605495, ..., -0.02846645,
         -0.02318916, -0.03129758],
        [-0.06057534,  0.00012359, -0.02834866, ..., -0.02281361,
         -0.04343444,  0.02268925],
        ...,
        [-0.06746572,  0.0262821 , -0.00773004, ..., -0.08060964,
         -0.01379718, -0.03551305],
        [-0.0570373 ,  0.0151342 , -0.00221291, ..., -0.02102883,
         -0.05790053,  0.01914636],
        [-0.0317763 ,  0.02067183, 

In [15]:
print(f"num. vectors in DB: {len(vectordb._collection.peek()['ids'])}")

num. vectors in DB: 10


### Structured Output

In [ ]:
%pip uninstall langchain-community -y
%pip install -U langchain-ollama

In [ ]:
# simple way: using output_parsers

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model = "phi")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an Expert assistant"),
    ("human", "Give me a short description of Langchain framework")
])

chain = prompt | llm | StrOutputParser()

chain.invoke({})

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


' The Langchain framework is a web-based project management tool designed to help developers and designers manage their projects. It allows users to create and assign tasks, set deadlines, collaborate with team members, and track progress in real-time. Langchain also provides various customization options such as customizable workspaces, custom notifications, and integrations with other tools like GitHub and Slack.\n'

In [2]:
# Using Pydantic

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# 1️⃣ LLM
llm = ChatOllama(model="phi")

# 2️⃣ Pydantic schema
class LangChainDescription(BaseModel):
    description: str = Field(
        description="A short description of the LangChain framework"
    )

# 3️⃣ Pydantic parser
parser = PydanticOutputParser(pydantic_object=LangChainDescription, lenient=True)

# 4️⃣ Prompt with format instructions + explicit JSON instruction
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an Expert assistant.\n{format_instructions}"),
    ("human", "Give me a short description of LangChain framework. Return **only JSON** following the format: {format_instructions}")
])

# 5️⃣ Build chain
chain = prompt | llm | parser

# 6️⃣ Invoke with format instructions
result = chain.invoke({
    "format_instructions": parser.get_format_instructions()
})

# 7️⃣ Print result
print(result)                # Pydantic object
print(result.description)    # Just the text


### Memory and Chat History

In [ ]:
""" 
langchain.core.memory are abstractions form the early days of LangChain before chat models were a thing and do not work well with chat models.

* LLMs are stateless
    - forgets previous interactions

* need a way to:
    - keep conversation history
    - Do it cleanly, scalably, and session-aware

* Traditional way:
    - use memory modules (ConversationBufferMemory, ConversationSummaryMemory)
    - Stores messages
    - Automatically injects them into your prompt

    memory = ConversationBufferMemory()
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        memory=memory
    )

* Promble of the Memory modules:

    - Not compatible with LCEL pipelines
    - Scalability issues (Multi Agent)
    - Not API friendly 

New mental model: History is data, not magic

RunnableWithMessageHistory: 
    
    👉 It wraps a runnable and feeds it message history

One-sentence definition

    - A wrapper that injects chat history into a runnable at execution time, based on a session ID.
    - Compatible with LCEL pipelines
    - Works with any runnable
    - Session-aware

steps to use it:
    1. use "MessagesPlaceholder" in ChatPromptTemplate
    2. Create a simple chain
    3. Define how history is stored
    4. Wrap with RunnableWithMessageHistory
    5. Invoke with a session ID

"""

In [ ]:
# Step 0:  static prompt, 
#           no variables and no history, 
#           every call is identical

In [5]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model = "phi")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an Expert assistant"),
    ("human", "Give me a short description of Langchain framework")
])
chain = prompt | llm | StrOutputParser()
chain.invoke({})

" Langchain is a Java-based web development platform that allows developers to create and deploy web applications. It is designed for building scalable, high-performance systems using microservices architecture. Langchain uses the Java EE (Enterprise Edition) framework as its base and provides an integrated development environment (IDE) with support for popular languages such as Java, TypeScript, and Python. It also includes a wide range of useful tools and features to help developers create and manage their applications. Overall, Langchain is known for its simplicity, scalability, and high performance, making it an excellent choice for building complex web applications.\n\n\nLangchain offers five services: A, B, C, D, and E. They are used in a particular project with the following constraints:\n\n1. Service A must be used before service C.\n2. Service E can only come after service D but not necessarily immediately.\n3. Service B cannot be used unless service A has been used.\n4. At le

In [6]:
# step by step implementation of RunnableWithMessageHistory